### Preparing our environment


#### I will install important libraries and create API keys for openAI and weather app

In [1]:
%pip install -q openai sentencepiece transformers


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
from dotenv import load_dotenv,find_dotenv
load_dotenv(dotenv_path="/Users/katka/AI AGENT/enviroments.env",override=True)
os.environ.get("OPENAI_API_KEY")
#os.environ.get("WEATHER")
print('API key loaded',os.environ.get("OPENAI_API_KEY") is not None)
#print('API key loaded',os.environ.get("WEATHER") is not None)

API key loaded True


In [3]:
import openai
import re
import os
import requests
from datetime import datetime,timedelta

Testing configuration of  OPEN AI 

In [4]:
from openai import OpenAI
client=OpenAI()

completion=client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            'role':'user',
            'content':"Write something short but funny"
        }
    ]
)

completion.choices[0].message.content

'Why don’t scientists trust atoms? Because they make up everything!'

Next, I need to create a class for a Travel Agent, who will perform specific actions and, for example, recalculate the budget.

In [5]:
class TravelAgent:
    def __init__(self,prompt,budget=1000):  # tu len pridam jednotlive self premenne
        self.system_prompt=prompt
        self.budget=budget
        self.messages=[
            {'role':'system','content': prompt},
            {'role':'system','content': f'The curent budget is ${self.budget}'}
        ]
        self.memory=[]
    
    def __call__(self, user_input):
        self.messages.append({'role':"user",'content': user_input})
        response=self.execute()
        self.messages.append({'role':'user','content': response})
        return response
    
    def execute(self):
        completion=client.chat.completions.create(
            model='gpt-4o-mini',
            messages=self.messages,
            temperature=0.7,
        )
        return completion.choices[0].message.content
    
    def add_memory(self,action,result):
        self.memory.append(f'Action: { action},Result: { result}')
        

    def set_budget(self,amount):
        self.budget=amount
        self.messages.append({'role':'system','content':f"The current budget is: ${self.budget}."})

    
    def get_budget(self):
        return self.budget



Now we will create some TOOLS

## Currency Tool

In [6]:
def currency(amount,from_currency,to_currency):
    url=f"https://api.exchangerate-api.com/v4/latest/{from_currency}"
    response=requests.get(url)
    data=response.json()
    rate=data['rates'][to_currency]
    converted_amount=amount*rate
    return round(converted_amount,2)


In [7]:
currency(100,"CZK","EUR")

3.97

## Weather Tool

In [8]:


def weather(location:str,days_ahead:int=0)-> str:
    load_dotenv(dotenv_path="/Users/katka/AI AGENT/enviroments.env",override=True)
    api_key=os.environ.get("OPENWEATHERMAP_API_KEY")
    
    if not api_key:
        raise Exception("Variable not set.")

    if days_ahead > 0:
        target_date = (datetime.now()+ timedelta(days=days_ahead)).strftime('%Y-%m-%d')
    else:
        target_date=None

    base_url="http://api.openweathermap.org/data/2.5/weather"

    params={
        "q":location,
        "appid":api_key,
        "units":"metric",
    }
    response=requests.get(base_url,params=params)
    if response.status_code==200:
        data=response.json()
        print(data)
        if target_date:
            return f"THe weather in {data['name']},on {target_date} is currently {data['weather'][0]['description']}"
        else:
            return f"The curent weather in {data['name']}, is currently {data['weather'][0]['description']}"
    else:
        return f"Error, {response.status_code}-{response.reason}"



    


In [9]:
weather('Bratislava',3)

{'coord': {'lon': 17.1067, 'lat': 48.1482}, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02d'}], 'base': 'stations', 'main': {'temp': 3.03, 'feels_like': -0.63, 'temp_min': 2.01, 'temp_max': 3.81, 'pressure': 1029, 'humidity': 66, 'sea_level': 1029, 'grnd_level': 1006}, 'visibility': 10000, 'wind': {'speed': 4.12, 'deg': 340}, 'clouds': {'all': 20}, 'dt': 1738573531, 'sys': {'type': 2, 'id': 2044188, 'country': 'SK', 'sunrise': 1738563422, 'sunset': 1738598023}, 'timezone': 3600, 'id': 3060972, 'name': 'Bratislava', 'cod': 200}


'THe weather in Bratislava,on 2025-02-06 is currently few clouds'

## Translation Tool

In [10]:
from transformers import pipeline


def translation(text,target_language):
    try:
        translator = pipeline("translation",model="facebook/m2m100_418M")
        translation = translator(text,src_lang="en",tgt_lang=target_language)[0]['translation_text']
        return translation
    except Exception as e:
        print(f"Error:{e}")
        return "Translation failed."



c:\Users\katka\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
from transformers import pipeline

def translation(text, target_language):
    try:
        # Initialize the translation pipeline with the model
        translator = pipeline("translation", model="facebook/m2m100_418M", device=-1)  # Use CPU

        # Ensure both `src_lang` and `tgt_lang` are set correctly
        translated = translator(
            text,
            src_lang='en',
            tgt_lang=target_language
        )
        return translated[0]['translation_text']
    except Exception as e:
        print(f"Error: {e}")
        return "Translation failed."


In [12]:
text="Dad, i love you."
language='fr'
translation(text,language)

Device set to use cpu


'Papa, je t’aime'

### Prompt

In [13]:
travel_agent_prompt="""You are Travel Agent Assistant mostly for girls and Women.
 Your role is to help others to plan trips by providing information about destination,weather ,currency and language translation etc.
 
 You run in a loop of Thought, Action, PAUSE and Observation.
 Use Thought for analyzing and reasoning user's request.
 Use Action to perform one of the available actions and then return PAUSE.
 Use Observation as result of running those actions.

 Your available actions are:

 1. currency:
 e.g., currency: 100,USD ,EUR
 Converts given amount from one currency to other.

 2. weather:
 e.g., weather: Bratislava , 3
 Gets the forecast for specified city and number of days ahead.

 3. translation:
 e.g., translation: Hello daddy, how are you?,cz
 Translates given text to the specified target language (cz for Chzech language)

 But always consider users budget when making suggestions.


 Example session:

 User: Iam planning trip to Zurich next week. What should I know if i have 1000 euros budget ?

 Thought: I should check the weather in Zurich for next week and provide some basic phrases.
 Action: weather: Zurich , 7
 PAUSE

 Observation: Zurich,cloudy, 20°C

 Thought: Now if i have the weather information , i should provide a simple phrase that are use in SWITZERLAND ..and its translation.
 Action: translation: Hello. How are you ?, DE
 PAUSE 

 Observation: Hallo, wie geht es dir?

 Thought: Now i need to check for currency..
 Action: currency: 1000,EUR,CHF
 PAUSE

 Observation: 937.09 CHF

 
 """

### We put our tools into dictionary

In [14]:
available_tools={
    'currency': currency,
    'weather': weather,
    'translation': translation,
}


## THE MAIN CHARACTER
 

In [15]:
def plan_trip(question,max_iter=5):
    agent=TravelAgent(travel_agent_prompt)

    # budget 
    budget_match=re.match(r'\$(\d+)',question)
    if budget_match:
        budget = int(budget_match.group(1))
        agent.set_budget(budget)
    
    next_prompt= question

    action_re = re.compile(r'^Action: (\w+): (.+)$')

    for i in range(max_iter):
        result=agent(next_prompt)
        print(result)

        actions=[action_re.match(a) for a in result.split('\n') if action_re.match(a)]

        if actions:
            action,action_input=actions[0].groups()

            if action not in available_tools:
                print("Unknown action:{action}")
                continue

            print(f"running {action}:{action_input} ")

            try:
                if action =="currency":
                    amount,from_currency,to_currency=action_input.split(',')
                    observation=available_tools[action](float(amount),from_currency.strip(),to_currency.strip())


                elif action=="weather":
                    location,*days=action_input.split(',')
                    days_ahead=int(days[0]) if days else 0
                    observation=available_tools[action](location.strip())

                elif action=="translation":
                    if ',' in action_input:
                        text,target_lang=action_input.rsplit(',',1)
                    else:
                        text,target_lang=action_input.rsplit(None,1)
                    observation=available_tools[action](text.strip(),target_lang.strip())
                
                print(f"Observation: {observation}")
                agent.add_memory(action,observation)
                next_prompt=f'Observation: {observation}'
            except Exception as e:
                print(f"Error: {e}")
                next_prompt=f'Error,unable to do this action:{action}.Please try again'
        else:
            return 

In [17]:
question="Hi, can you tell me what I should do if I'm planning to go to London with a budget of 2,000 euros?"
plan_trip(question)

Thought: I should check the weather in London for the upcoming week and provide some basic phrases in English. Additionally, I need to convert the budget from euros to pounds to better understand the available amount for the trip. 

Action: weather: London, 7
PAUSE
running weather:London, 7 
{'coord': {'lon': -0.1257, 'lat': 51.5085}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 4.36, 'feels_like': 4.36, 'temp_min': 2.29, 'temp_max': 5.05, 'pressure': 1025, 'humidity': 91, 'sea_level': 1025, 'grnd_level': 1021}, 'visibility': 10000, 'wind': {'speed': 1.03, 'deg': 0}, 'clouds': {'all': 100}, 'dt': 1738572974, 'sys': {'type': 2, 'id': 2075535, 'country': 'GB', 'sunrise': 1738568138, 'sunset': 1738601580}, 'timezone': 0, 'id': 2643743, 'name': 'London', 'cod': 200}
Observation: The curent weather in London, is currently overcast clouds
Thought: Now that I have the weather information for London, I should 

Device set to use cpu


Error: 'EN'
Observation: Translation failed.
Thought: Since the translation failed, I will try to provide a common English phrase without performing the action, as English is the primary language spoken in London. Additionally, I will proceed with converting the budget from euros to pounds.

Action: currency: 2000,EUR,GBP
PAUSE
running currency:2000,EUR,GBP 
Observation: 1670.0
Thought: I now have the currency conversion result, indicating that 2,000 euros is approximately 1,670 pounds. I have the weather information and a common phrase in English. I should summarize the information for the user.

Action: Provide a summary of the trip information for London. 
PAUSE
